In [1]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import cv2
import os
import numpy as np

In [3]:
from sklearn.preprocessing import LabelBinarizer

In [10]:
# load model
model = load_model('model_LSTM2D.h5')
# summarize model.
model.summary()


Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_81 (Conv2D)           (None, 32, 97, 1497)      544       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 10, 32, 1497)      0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 8, 30, 32)         431168    
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 6, 28, 32)         9248      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 3, 14, 32)         0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 1, 12, 32)         9248      
_________________________________________________________________
reshape_5 (Reshape)          (None, 12, 32)          

In [11]:
print("[INFO] loading images...")

# imagePaths = list(paths.list_images(args["dataset"]))
imagePaths = ["Capture", "Pointing", "ZoomIn", "ZoomOut"]
data = []
labels = []
path = "Testing/"


# names = [i for i in os.listdir(path)]
names = ["Test3"]
for name in names:
    gestures = [i for i in os.listdir(path + name)]
#     gestures = ["ZoomIn"]
    for gesture  in gestures:
        folders = [i for i in os.listdir(path + name + "/" + gesture)]
        for folder in folders:
            photos = [i for i in os.listdir(path + name + "/" + gesture + "/" + folder)]
            arr = np.zeros((100,100))
            for photo in photos:
                image = cv2.imread(path + name + "/" + gesture + "/" + folder + "/" + photo)
                arr = np.concatenate((arr, image[:,:,0]), axis=1)
            data.append(arr[:,100:])
            labels.append(gesture)
            
            
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# print(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(data.shape)
print(labels.shape)


[INFO] loading images...
(8, 100, 1500)
(8, 4)


In [12]:
data = data.reshape([len(data),1,100, 1500])
print(data.shape)
print(labels.shape)
print(lb.classes_)
print(lb.transform(lb.classes_))

(8, 1, 100, 1500)
(8, 4)
['Capture' 'Pointing' 'ZoomIn' 'ZoomOut']
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [14]:
# evaluate the model
score = model.evaluate(data, labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 75.00%
